In [1]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [18]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import VanGame.utils as utils

In [5]:
sequence = sorted([p for p in CELLS])

In [6]:
a = np.array([[1]*39])
b = [500]

In [7]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(200,activation='relu', input_shape=(39,)))
    model.add(layers.Dense(500, activation='relu'))
    model.add(layers.Dense(500, activation='relu'))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [257]:
model = build_model()

In [107]:
try:
    model = load_model('VanGame/my_model.h5')
except:
    print("none model found")
    model = build_model()

In [14]:
!ls 'rec'

'[1win]1556539493.0caoredjiba.txt'  '[2zzz]1556539494.0caoredjiba.txt'
'[1win]1556539499.0caoredjiba.txt'  '[2zzz]1556539497.0caoredjiba.txt'
'[1win]1556539502.0caoredjiba.txt'  '[2zzz]1556539500.0caoredjiba.txt'
'[1win]1556539504.0caoredjiba.txt'  '[2zzz]1556539508.0caoredjiba.txt'
'[1win]1556539506.0caoredjiba.txt'  '[2zzz]1556539509.0caoredjiba.txt'
'[2zzz]1556539486.0caoredjiba.txt'  '[2zzz]1556539510.0caoredjiba.txt'
'[2zzz]1556539487.0caoredjiba.txt'  '[2zzz]1556539511.0caoredjiba.txt'
'[2zzz]1556539488.0caoredjiba.txt'  '[2zzz]1556539512.0caoredjiba.txt'
'[2zzz]1556539489.0caoredjiba.txt'  '[2zzz]1556539513.0caoredjiba.txt'
'[2zzz]1556539491.0caoredjiba.txt'  '[2zzz]1556539514.0caoredjiba.txt'


In [24]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [25]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [26]:
def load_logs():
    datapath = join(os.getcwd(),'rec/')
    files = bash_call(['ls', datapath])
    return files

In [258]:
def PG_log(filename, condition=None, value_check=False):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    #print(filename)
    
    states, color = utils.load_l('rec/'+filename)
    
    train = []
    value = []
    
    for s in states:
        board = []
        # add board
        for cord in sequence:
            board.append(s['board'][cord])
        # color 
        board.append(color)
        board.append(s['d_heur'])
        train.append(board)
        value.append(s['rew'])
        
    if value_check:
        print(value)
        
    value.reverse()
    decay = 0.95
    curr_decay = 0.95
    for i in range(1, len(value)):
        value[i] += value[i-1] * curr_decay
        curr_decay *= decay
    
    value.reverse()
    
    if value_check:
        print(value)
    
    return train, value

In [259]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')


In [260]:
def save_model():
    model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('trained_model', 'wb') as f:
        pickle.dump(model.get_weights(), f)

In [261]:
def train(save=True):
    logfs = load_logs()
    train_set = []
    target_set = []
    
    for logf in logfs:
        train, value = PG_log(logf)
        if train:
            #print(len(train), len(value))
            #print(train[-1])
            train_set += train
            target_set += value

    model.fit(np.array(train_set), target_set, epochs=3, batch_size=1)
    if save:
        print("saved")
        save_model()   

In [145]:
train()

[1win]1556549315.0caoredjiba.txt
[1win]1556549321.0caoredjiba.txt
[2zzz]1556549305.0caoredjiba.txt
[2zzz]1556549307.0caoredjiba.txt
[2zzz]1556549308.0caoredjiba.txt
[2zzz]1556549309.0caoredjiba.txt
[2zzz]1556549311.0caoredjiba.txt
Epoch 1/3
304/304 [==============================] - 2s 6ms/step - loss: 4546.4440
Epoch 2/3
304/304 [==============================] - 2s 6ms/step - loss: 3489.6866
Epoch 3/3
304/304 [==============================] - 2s 7ms/step - loss: 2265.0692
saved


In [262]:
save_model()

In [106]:
del model

In [263]:
model.predict(np.array([[1]*39,[0.5]*39]))

array([[-0.01214426],
       [-0.00607213]], dtype=float32)

In [52]:
def clean_log(logs):
    !rm '/Users/xinyaoniu/Documents/COMP30024-AI/Project2/rec/' *

In [53]:
clean_log(logfs)

rm: cannot remove '/Users/xinyaoniu/Documents/COMP30024-AI/Project2/rec/': Is a directory
rm: cannot remove '__pycache__': Is a directory


In [110]:
os.getcwd()

'/Users/xinyaoniu/Documents/COMP30024-AI/Project2'

In [146]:
!rm -v ./rec/*

removed './rec/[1win]1556549315.0caoredjiba.txt'
removed './rec/[1win]1556549321.0caoredjiba.txt'
removed './rec/[2zzz]1556549305.0caoredjiba.txt'
removed './rec/[2zzz]1556549307.0caoredjiba.txt'
removed './rec/[2zzz]1556549308.0caoredjiba.txt'
removed './rec/[2zzz]1556549309.0caoredjiba.txt'
removed './rec/[2zzz]1556549311.0caoredjiba.txt'


In [153]:
def full_train_cycle(n):
    for i in range(n):
        !python3 -m referee VanGame HardCode HardCode -l logtry -v 0
    train()
    !rm -v ./rec/*

In [265]:
for i in range(5):
    full_train_cycle(10)

* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/3
253/253 [==============================] - 2s 6ms/step - loss: 14538.7964
Epoch 2/3
253/253 [==============================] - 2s 7ms/step

In [226]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
md = Sequential()
#add model layers
md.add(Conv2D(1, kernel_size=3, activation='relu', input_shape=(28,28,1), use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
md.add(Flatten())
md.compile(optimizer='adam', loss='mse')

In [227]:
for i in md.get_weights():
    print(i.shape)

(3, 3, 1, 1)


In [228]:
md.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 26, 26, 1)         9         
_________________________________________________________________
flatten_9 (Flatten)          (None, 676)               0         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [229]:
md.predict(np.array([[[[1]*28]*28]]).reshape(1,28,28,1))

array([[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.

In [244]:
asdf = md.get_weights()[0].reshape(3,3)

In [230]:
wds = md.get_weights()

In [231]:
convp = wds[0]

In [232]:
convp.shape

(3, 3, 1, 1)

In [233]:
c = convp.reshape(1, 3, 3, 1)

In [234]:
d = c[0].reshape(3,3)

In [235]:
import scipy.signal as signal

In [236]:
a = np.array([[[1]*28]*28]).reshape(28,28)

In [245]:
e = signal.convolve2d(a, asdf, mode='valid')

In [246]:
e

array([[-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036],
       [-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036],
       [-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
      

In [206]:
signal.convolve(e, )

ValueError: volume and kernel should have the same dimensionality